In [1]:
import numpy as np 
import pandas as pd
import string

In [2]:
pd.set_option('display.max_colwidth', None)

## Data processing

In [3]:
!pwd

/Users/yinghu/Documents/GitHub/fourthBrain/glg-project/week_11_models


In [4]:
ner_dataset = pd.read_csv('/Users/yinghu/Documents/GitHub/fourthBrain/glg-project/datasets/ner_dataset.csv', 
    encoding='latin1')

Fill the missing `Sentence #`

In [5]:
ner_dataset['Sentence #'] = ner_dataset['Sentence #'].str.replace('Sentence:', '')
ner_dataset = ner_dataset.fillna(method='ffill')

In [17]:
ner_dataset['Sentence #'] = ner_dataset['Sentence #'].astype(int)

In [18]:
ner_dataset.dtypes

Sentence #     int64
Word          object
POS           object
Tag           object
dtype: object

#### Create the `sentences_df`

In [7]:
sentences_df = ner_dataset.groupby('Sentence #')['Word'].apply(lambda x: x.str.cat(sep=' '))
sentences_df = sentences_df.reset_index()
sentences_df = sentences_df.rename(columns={'Word': 'Sentences'})

Inspect Sentence 8411. It only contains the word "The". 

In [9]:
sentences_df.iloc[8411]

Sentence #    8412
Sentences      The
Name: 8411, dtype: object

In [10]:
sentences_df = sentences_df.drop(labels=[8411], axis=0)
sentences_df = sentences_df.reset_index()
sentences_df = sentences_df.drop(columns='index')

#### Lower Casing

In [19]:
sentences_df['sent_lower'] = sentences_df['Sentences'].str.lower()

#### Removal of punctuations

In [20]:
PUNCT_TO_REMOVE = string.punctuation
def remove_punctuation(sentence):
    return sentence.translate(str.maketrans('', '', PUNCT_TO_REMOVE))

sentences_df['sent_wo_punc'] = sentences_df['Sentences'].apply(lambda sentence: remove_punctuation(sentence))

In [21]:
sentences_df.head()

,Sentence #,Sentences,sent_lower,sent_wo_punc
0,1,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country .,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country
1,2,"Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" bush number one terrorist "" and "" stop the bombings . """,Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as Bush Number One Terrorist and Stop the Bombings
2,3,They marched from the Houses of Parliament to a rally in Hyde Park .,they marched from the houses of parliament to a rally in hyde park .,They marched from the Houses of Parliament to a rally in Hyde Park
3,4,"Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .","police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .",Police put the number of marchers at 10000 while organizers claimed it was 100000
4,5,The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .,the protest comes on the eve of the annual conference of britain 's ruling labor party in the southern english seaside resort of brighton .,The protest comes on the eve of the annual conference of Britain s ruling Labor Party in the southern English seaside resort of Brighton


In [22]:
sentences_df['sent_wo_punc_lower'] = sentences_df['sent_wo_punc'].str.lower()

#### Removal of stopwords 

In [23]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')
nltk.download('averaged_perceptron_tagger')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/yinghu/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /Users/yinghu/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /Users/yinghu/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/yinghu/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package punkt to /Users/yinghu/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [24]:
from nltk.corpus import stopwords 

In [35]:
STOPWORDS = set(stopwords.words('english'))
def remove_stopwords(sentence):
    return ' '.join([word for word in sentence.split() if word not in STOPWORDS])

sentences_df['sent_wo_stop'] = sentences_df['sent_wo_punc_lower']\
                            .apply(lambda sentence: remove_stopwords(sentence))
sentences_df.head()


,Sentence #,Sentences,sent_lower,sent_wo_punc,sent_wo_punc_lower,sent_lemmatized,sent_lemmatized_lower,sent_wo_stop
0,1,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country .,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country,Thousands of demonstrator have march through London to protest the war in Iraq and demand the withdrawal of British troop from that country .,thousands of demonstrator have march through london to protest the war in iraq and demand the withdrawal of british troop from that country .,thousands demonstrators marched london protest war iraq demand withdrawal british troops country
1,2,"Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" bush number one terrorist "" and "" stop the bombings . """,Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as Bush Number One Terrorist and Stop the Bombings,families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as bush number one terrorist and stop the bombings,"Families of soldier kill in the conflict join the protester who carry banner with such slogan a "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldier kill in the conflict join the protester who carry banner with such slogan a "" bush number one terrorist "" and "" stop the bombings . """,families soldiers killed conflict joined protesters carried banners slogans bush number one terrorist stop bombings
2,3,They marched from the Houses of Parliament to a rally in Hyde Park .,they marched from the houses of parliament to a rally in hyde park .,They marched from the Houses of Parliament to a rally in Hyde Park,they marched from the houses of parliament to a rally in hyde park,They march from the Houses of Parliament to a rally in Hyde Park .,they march from the houses of parliament to a rally in hyde park .,marched houses parliament rally hyde park
3,4,"Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .","police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .",Police put the number of marchers at 10000 while organizers claimed it was 100000,police put the number of marchers at 10000 while organizers claimed it was 100000,"Police put the number of marcher at 10,000 while organizer claim it be 1,00,000 .","police put the number of marcher at 10,000 while organizer claim it be 1,00,000 .",police put number marchers 10000 organizers claimed 100000
4,5,The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .,the protest comes on the eve of the annual conference of britain 's ruling labor party in the southern english seaside resort of brighton .,The protest comes on the eve of the annual conference of Britain s ruling Labor Party in the southern English seaside resort of Brighton,the protest comes on the eve of the annual conference of britain s ruling labor party in the southern english seaside resort of brighton,The protest come on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .,the protest come on the eve of the annual conference of britain 's ruling labor party in the southern eng

#### Lemmatization 

In [36]:
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer

In [37]:
lemmatizer = WordNetLemmatizer()
wordnet_map = {'N':wordnet.NOUN, 'V':wordnet.VERB, 'J':wordnet.ADJ, 'R':wordnet.ADV}
def lemmatize_words(sentence):
    pos_tagged_text = nltk.pos_tag(sentence.split())
    return ' '.join([lemmatizer.lemmatize(word, wordnet_map.get(pos[0], wordnet.NOUN))
                    for word, pos in pos_tagged_text])

sentences_df['sent_lemmatized'] = sentences_df['Sentences'].apply(lambda sentence: lemmatize_words(sentence))

In [38]:
sentences_df.head()

,Sentence #,Sentences,sent_lower,sent_wo_punc,sent_wo_punc_lower,sent_lemmatized,sent_lemmatized_lower,sent_wo_stop
0,1,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country .,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country,Thousands of demonstrator have march through London to protest the war in Iraq and demand the withdrawal of British troop from that country .,thousands of demonstrator have march through london to protest the war in iraq and demand the withdrawal of british troop from that country .,thousands demonstrators marched london protest war iraq demand withdrawal british troops country
1,2,"Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" bush number one terrorist "" and "" stop the bombings . """,Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as Bush Number One Terrorist and Stop the Bombings,families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as bush number one terrorist and stop the bombings,"Families of soldier kill in the conflict join the protester who carry banner with such slogan a "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldier kill in the conflict join the protester who carry banner with such slogan a "" bush number one terrorist "" and "" stop the bombings . """,families soldiers killed conflict joined protesters carried banners slogans bush number one terrorist stop bombings
2,3,They marched from the Houses of Parliament to a rally in Hyde Park .,they marched from the houses of parliament to a rally in hyde park .,They marched from the Houses of Parliament to a rally in Hyde Park,they marched from the houses of parliament to a rally in hyde park,They march from the Houses of Parliament to a rally in Hyde Park .,they march from the houses of parliament to a rally in hyde park .,marched houses parliament rally hyde park
3,4,"Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .","police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .",Police put the number of marchers at 10000 while organizers claimed it was 100000,police put the number of marchers at 10000 while organizers claimed it was 100000,"Police put the number of marcher at 10,000 while organizer claim it be 1,00,000 .","police put the number of marcher at 10,000 while organizer claim it be 1,00,000 .",police put number marchers 10000 organizers claimed 100000
4,5,The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .,the protest comes on the eve of the annual conference of britain 's ruling labor party in the southern english seaside resort of brighton .,The protest comes on the eve of the annual conference of Britain s ruling Labor Party in the southern English seaside resort of Brighton,the protest comes on the eve of the annual conference of britain s ruling labor party in the southern english seaside resort of brighton,The protest come on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .,the protest come on the eve of the annual conference of britain 's ruling labor party in the southern eng

In [39]:
sentences_df['sent_lemmatized_lower'] = sentences_df['sent_lemmatized'].str.lower()

In [40]:
def remove_punct_stopwords(sentence):
    sentence = remove_punctuation(sentence)
    return remove_stopwords(sentence)

sentences_df['sent_lemmatized_wo_punct_sw'] = sentences_df['sent_lemmatized_lower'].apply(lambda sentence: remove_punct_stopwords(sentence))

In [41]:
sentences_df

,Sentence #,Sentences,sent_lower,sent_wo_punc,sent_wo_punc_lower,sent_lemmatized,sent_lemmatized_lower,sent_wo_stop,sent_lemmatized_wo_punct_sw
0,1,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country .,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country .,Thousands of demonstrators have marched through London to protest the war in Iraq and demand the withdrawal of British troops from that country,thousands of demonstrators have marched through london to protest the war in iraq and demand the withdrawal of british troops from that country,Thousands of demonstrator have march through London to protest the war in Iraq and demand the withdrawal of British troop from that country .,thousands of demonstrator have march through london to protest the war in iraq and demand the withdrawal of british troop from that country .,thousands demonstrators marched london protest war iraq demand withdrawal british troops country,thousands demonstrator march london protest war iraq demand withdrawal british troop country
1,2,"Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as "" bush number one terrorist "" and "" stop the bombings . """,Families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as Bush Number One Terrorist and Stop the Bombings,families of soldiers killed in the conflict joined the protesters who carried banners with such slogans as bush number one terrorist and stop the bombings,"Families of soldier kill in the conflict join the protester who carry banner with such slogan a "" Bush Number One Terrorist "" and "" Stop the Bombings . ""","families of soldier kill in the conflict join the protester who carry banner with such slogan a "" bush number one terrorist "" and "" stop the bombings . """,families soldiers killed conflict joined protesters carried banners slogans bush number one terrorist stop bombings,families soldier kill conflict join protester carry banner slogan bush number one terrorist stop bombings
2,3,They marched from the Houses of Parliament to a rally in Hyde Park .,they marched from the houses of parliament to a rally in hyde park .,They marched from the Houses of Parliament to a rally in Hyde Park,they marched from the houses of parliament to a rally in hyde park,They march from the Houses of Parliament to a rally in Hyde Park .,they march from the houses of parliament to a rally in hyde park .,marched houses parliament rally hyde park,march houses parliament rally hyde park
3,4,"Police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .","police put the number of marchers at 10,000 while organizers claimed it was 1,00,000 .",Police put the number of marchers at 10000 while organizers claimed it was 100000,police put the number of marchers at 10000 while organizers claimed it was 100000,"Police put the number of marcher at 10,000 while organizer claim it be 1,00,000 .","police put the number of marcher at 10,000 while organizer claim it be 1,00,000 .",police put number marchers 10000 organizers claimed 100000,police put number marcher 10000 organizer claim 100000
4,5,The protest comes on the eve of the annual conference of Britain 's ruling Labor Party in the southern English seaside resort of Brighton .,the protest comes on the eve of the annual conference of britain 's ruling labor party in the southern english seaside resort of brighton .,The protest comes on the eve of the annual conference of Britain s ruling Labor Party in the southern English seaside resort of Brighton,the protest comes on the eve of the annual conference of br

In [42]:
sentences_df.isna().sum()

Sentence #                     0
Sentences                      0
sent_lower                     0
sent_wo_punc                   0
sent_wo_punc_lower             0
sent_lemmatized                0
sent_lemmatized_lower          0
sent_wo_stop                   0
sent_lemmatized_wo_punct_sw    0
dtype: int64

### Save `sentences_df` as csv files

In [70]:
# reduced_df.to_csv('reduced_df.csv')
# sentences_df.to_csv('sentences_v1.csv')

## LDA Model

In [43]:
from gensim import models, matutils
from sklearn.feature_extraction.text import CountVectorizer

#### LDA model + BOW 

In [44]:
shortened_sent = sentences_df['sent_lemmatized_wo_punct_sw'].copy()

In [45]:
bow_vectorizer = CountVectorizer()
shortened_sent_bow = bow_vectorizer.fit_transform(shortened_sent)

In [46]:
shortened_sent_bow.shape

(47958, 26981)

In [47]:
type(shortened_sent_bow)

scipy.sparse._csr.csr_matrix

In [48]:
id2word = {idx: word for word, idx in bow_vectorizer.vocabulary_.items()}
corpus = shortened_sent_bow.transpose()
corpus = matutils.Sparse2Corpus(corpus)

In [49]:
lad = models.LdaModel(corpus, num_topics=3, id2word=id2word)
lad.print_topics()

[(0,
  '0.036*"say" + 0.016*"kill" + 0.012*"people" + 0.011*"official" + 0.011*"attack" + 0.010*"two" + 0.009*"police" + 0.008*"force" + 0.007*"us" + 0.007*"militant"'),
 (1,
  '0.011*"say" + 0.009*"year" + 0.006*"last" + 0.006*"one" + 0.004*"report" + 0.004*"world" + 0.004*"new" + 0.004*"day" + 0.004*"week" + 0.004*"first"'),
 (2,
  '0.022*"say" + 0.011*"president" + 0.010*"mr" + 0.010*"us" + 0.008*"government" + 0.007*"united" + 0.007*"country" + 0.006*"minister" + 0.006*"new" + 0.006*"iran"')]

## Word embedding

### word2vec for future reference

In [ ]:
## Download the full model 
# import gensim.downloader as api 
# wv = api.load('word2vec-google-news-300')

## Clustering Model 

### K-Mean with bag of words

In [58]:
from sklearn.pipeline import Pipeline
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.metrics import silhouette_score, calinski_harabasz_score, davies_bouldin_score

Use bag of words to encode shortened sentences

In [33]:
vectorizer = CountVectorizer()
shortened_sentences_bow = vectorizer.fit_transform(sentences_df['Shortened Sentences'])
print(shortened_sentences_bow.shape)

(47959, 27706)


Fit and evaluate K-Mean model (true labels are unknown)

In [57]:
for n_cluster in range(3,15):
    kmeans_model = KMeans(n_clusters=n_cluster, random_state=1).fit(shortened_sentences_bow)
    labels = kmeans_model.labels_

    # print(f'Calinski-Harabasz Index: {calinski_harabasz_score(sample_data, labels)}')
    # print(f'Davies-Bouldin Index: {davies_bouldin_score(sample_data, labels)}')
    print(f'n_clusters = {n_cluster}: Silhouette Coefficient: {silhouette_score(shortened_sentences_bow, labels)}')

n_clusters = 3: Silhouette Coefficient: 0.010068379210426638
n_clusters = 4: Silhouette Coefficient: 0.011919093397266331
n_clusters = 5: Silhouette Coefficient: 0.013364127035644687
n_clusters = 6: Silhouette Coefficient: 0.012966342284531843
n_clusters = 7: Silhouette Coefficient: 0.032791462910272025
n_clusters = 8: Silhouette Coefficient: 0.028764928833763415
n_clusters = 9: Silhouette Coefficient: 0.014308675488758934
n_clusters = 10: Silhouette Coefficient: 0.013340042737080206
n_clusters = 11: Silhouette Coefficient: 0.011698526249829278
n_clusters = 12: Silhouette Coefficient: 0.012158417380356865
n_clusters = 13: Silhouette Coefficient: 0.013536046907808304
n_clusters = 14: Silhouette Coefficient: 0.009958963172837274


### K-Mean with TF-IDF 

In [59]:
vectorizer_tfidf = TfidfVectorizer()
shortened_sentences_tfidf = vectorizer_tfidf.fit_transform(sentences_df['Shortened Sentences'])

In [61]:
shortened_sentences_tfidf.shape

(47959, 27706)

In [62]:
for n_cluster in range(3,15):
    kmeans_model = KMeans(n_clusters=n_cluster, random_state=1).fit(shortened_sentences_tfidf)
    labels = kmeans_model.labels_

    # print(f'Calinski-Harabasz Index: {calinski_harabasz_score(sample_data, labels)}')
    # print(f'Davies-Bouldin Index: {davies_bouldin_score(sample_data, labels)}')
    print(f'n_clusters = {n_cluster}: Silhouette Coefficient: {silhouette_score(shortened_sentences_tfidf, labels)}')

n_clusters = 3: Silhouette Coefficient: 0.001727999414975458
n_clusters = 4: Silhouette Coefficient: 0.0022900932816321737
n_clusters = 5: Silhouette Coefficient: 0.0025437715733283026
n_clusters = 6: Silhouette Coefficient: 0.003272915172624837
n_clusters = 7: Silhouette Coefficient: 0.003644550697878073
n_clusters = 8: Silhouette Coefficient: 0.003361256118098299
n_clusters = 9: Silhouette Coefficient: 0.003750177526948939
n_clusters = 10: Silhouette Coefficient: 0.004631039850651867
n_clusters = 11: Silhouette Coefficient: 0.004013381132652468
n_clusters = 12: Silhouette Coefficient: 0.0042527648207261876
n_clusters = 13: Silhouette Coefficient: 0.005017270655819071
n_clusters = 14: Silhouette Coefficient: 0.005285629825189062
